In [31]:
import requests
from bs4 import BeautifulSoup
import string
import time
import re
import string
import os
from collections import Counter,OrderedDict
import nltk
from nltk.tokenize import word_tokenize
import glob
import pickle
import math
import operator

In [32]:
def isEnglish(s):
    return s.translate(None, string.punctuation).isalnum()

In [43]:
visitedLinks = []
depth = 1
collectedLinks = []
finalLinks = {}
raw_html = {}
inverted_unigram_dict = dict()
unigram_termcount = {}
unigram_corpus_count = 0
path = 'corpus'
smoothening_factor = 1500

In [34]:
def casefolding(data):
    return data.lower()

def punctuationHandling(data):
    regex = r"(?<!\d)[.,;:*!\"\'#$%&()+/<=>?@[\]_^`~{|}∑α](?!\d)"
    data = re.sub(regex, "", data, 0)
    regex = r"[;:*!\"\'#$%&()+/<=>?@[\]_^`~{|}]"
    data = re.sub(regex, "", data, 0)
#     data = data.strip(string.punctuation)
    return data

def removeWhitespace(data):
    data = ' '.join(data.split())
    return data

In [35]:
def createCorpusFile(heading,maincontent):
    fo = open("corpus/"+str(heading)+".txt", "w")
    fo.write(maincontent)
    # Close opend file
    fo.close()

In [36]:
def parseDocs():
    for filename in glob.glob("test-collection/cacm/*.html"):
        fo = open(filename, "r")
        heading = os.path.basename(filename).split(".")[0]
        data = fo.read()
        maincontent = BeautifulSoup(data, "lxml").text
        maincontent = casefolding(maincontent)
        maincontent = punctuationHandling(maincontent)
        maincontent = removeWhitespace(maincontent)
        createCorpusFile(heading,maincontent)

In [37]:
def removeDuplicates(mainList, fetchedList):
    links = []
    for i in fetchedList:
        if i not in mainList:
            links.append(i)
    return links        

In [38]:
def addToResultList(totalLinkList,links,depth):
    for i in links:
        totalLinkList.append([link_prefix + i,depth])
    return totalLinkList

In [39]:
def createIndexDict(file, ngram_dict, inverted_dict):
    for key,value in ngram_dict.items():
        if (inverted_dict.get(key)):
            inverted_dict.get(key).append((os.path.basename(file).split(".")[0],value))
        else:
            inverted_dict[key] = [(os.path.basename(file).split(".")[0],value)]
    return inverted_dict

In [40]:
parseDocs()

In [44]:
# for filename in for file in glob.glob("*.jpg")os.listdir(path):
for filename in glob.glob("corpus/*.txt"):
#     print(filename)
    fo = open(filename, "r")
    data = fo.read()
    tokens = nltk.word_tokenize(data)
    unigramlist = nltk.word_tokenize(data)
#     unigram_termcount[os.path.basename(filename).split(".")[0]] = len(set(unigramlist))
#     unigram_corpus_count = unigram_corpus_count + len(set(unigramlist))
    unigram_termcount[os.path.basename(filename).split(".")[0]] = len(unigramlist)
    unigram_corpus_count = unigram_corpus_count + len(unigramlist)
    unigram_dict = Counter(unigramlist)
    inverted_unigram_dict = createIndexDict(filename, unigram_dict, inverted_unigram_dict)
    fo.close()

In [45]:
def getInvertedListCount(index_list):
    invertedlist_count = {}
    for key,val in index_list.items():
        invertedlist_count[key] = [len(val),val]
    return invertedlist_count

In [46]:
unigram_invertedlist_count = getInvertedListCount(inverted_unigram_dict)